## 9.4 다양한 웹 소스에서 자동으로 정보 검색하기
### 유튜브 영상 요약하고 다국어 번역하기

* 책 312~317 쪽

<img src='https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/cover.png'
     alt="NLP와 LLM 실전 가이드(한빛미디어)"
     style="border: 3px solid gray; box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.3); border-radius: 10px; width: 300px;"   width='300'>


* 저자:  
    - [Lior Gazit](https://www.linkedin.com/in/liorgazit).  
    - [Meysam Ghaffari](https://www.linkedin.com/in/meysam-ghaffari-ph-d-a2553088/).
* 역자:
    - [박조은](https://github.com/corazzon)
* 이 노트북은 다음의 책에서 소개하는 내용입니다.
    - 역서 : NLP와 LLM 실전 가이드(한빛미디어)
    - 원서 : [Mastering NLP from Foundations to LLMs](https://www.amazon.com/dp/1804619183)

**이 노트북의 목적:**  
보고 싶은 유튜브 영상을 직접 시청할 필요 없이 요약하고, 원하는 방식으로 편집하세요.  
이 노트북에서는 인기 있는 TED 강연 중 하나를 선택해 요약하고, 한국어로 번역한 후, 이를 목록 형식으로 편집하여 제공합니다.

**요구 사항:**  
* Colab에서 실행할 때, 다음 런타임 설정을 사용하세요: `Python3, CPU`  
* 이 코드는 OpenAI의 API를 LLM으로 선택하므로, 유료 **API 키**가 필요합니다.  

colab 실습 :
https://github.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/Mastering-NLP-from-Foundations-to-LLMs/blob/main/Chapter9_notebooks/Ch9_Retrieve_Content_from_a_YouTube_Video_and_Summarize.ipynb)  


원서 Colab 실습:
https://github.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs   
<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter9_notebooks/Ch9_Retrieve_Content_from_a_YouTube_Video_and_Summarize.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

>*```면책 사항: 이 노트북에서 제시된 내용과 아이디어는 전적으로 저자의 것이며, 저자의 고용주의 견해나 지적 재산을 대표하지 않습니다.```*

Install:

* 원서의 embedchain이 mem0(https://mem0.ai/ )로 변경되었습니다. mem0는 코드는 오픈소스로 제공되나 API 비용이 별도로 부과됩니다. OpenAI API도 비용이 발생하나, 번거롭게 여러 API Key를 발급받지 않도록 OpenAI API의 해당 도서의 번역시점에서 가장 저렴한 gpt-4o-mini 모델 사용으로 대체합니다.

 * 역자 업데이트 : 원서의 embedchain이 mem0로 이름이 변경되었습니다.
 * 원서에는 embedchain 를 사용하지만 유료 API 로 변경이 되어 랭체인으로 대체합니다.

In [ ]:
# 필요한 주요 라이브러리 설치
!pip install -Uq langchain langchain_openai langchain_community youtube-transcript-api

Imports:

In [ ]:
import os
import textwrap
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

### 코드 설정

OpenAI API key:  
**문자열 형태로 아래 "..."에 OpenAI에서 발급받은 key를 입력해 주세요!**  


Colab 보안 비밀 설정은 왼쪽 열쇠 모양의 아이콘을 클릭하면 나옵니다.
<img src="https://i.imgur.com/7P383n4.png" width="500">

In [ ]:
os.environ["OPENAI_API_KEY"] = "..."

# Colab 에서는 보안 비밀키 설정을 통해 API 키를 매번 입력하지 않고 아래와 같이 관리할 수 있습니다.
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    if not os.environ["OPENAI_API_KEY"]:
        raise ValueError("Colab의 보안 비밀 설정에서 'OPENAI_API_KEY'를 찾을 수 없습니다.")
except ImportError:
    print("Colab 환경이 아니므로, 'api_key' 변수에 직접 값을 할당해야 합니다.")

#### Youtube Video URL

In [ ]:
# 유튜브 비디오 URL
video_url = "https://www.youtube.com/watch?v=8KkKuTCFvzI&ab_channel=TED"

### 검색하고 원본 문서 확인  
이 예시에서는 RAG에 단일 문서만을 컨텍스트로 제공합니다.
첫 1000자를 확인해 봅시다.

In [ ]:
# 유튜브 비디오 ID 추출
video_id = video_url.split("v=")[-1]

# 자막 추출
try:
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id,
                                                          languages=['en'])
    english_text = " ".join([entry['text'] for entry in transcript_list])
    print(english_text[:1000])
except Exception as e:
    print(f"자막을 가져오는 중 오류 발생: {e}")
    english_text = ""

What keeps us healthy and happy as we go through life? If you were going to invest now in your future best self, where would you put your time
and your energy? There was a recent survey of millennials asking them what their
most important life goals were, and over 80 percent said that a major life goal for them
was to get rich. And another 50 percent
of those same young adults said that another major life goal was to become famous. (Laughter) And we're constantly told
to lean in to work, to push harder and achieve more. We're given the impression that these
are the things that we need to go after in order to have a good life. Pictures of entire lives, of the choices that people make
and how those choices work out for them, those pictures
are almost impossible to get. Most of what we know about human life we know from asking people
to remember the past, and as we know, hindsight
is anything but 20/20. We forget vast amounts
of what happens to us in life, and sometimes memory
is downrigh

## 검토, 요약, 다국어 번역

In [ ]:
# 모델 설정
llm = ChatOpenAI(model="gpt-4o-mini")

summary_prompt_template = """
전체 내용을 검토하고, 4문장 길이의 영어로 요약한 다음 한국어, 러시아어, 독일어로 번역해 주세요.
요약이 내용과 일치하는지 확인하세요.
답변의 영어와 한국어 부분 사이에 '\n----\n' 문자열을 넣어주세요.
답변의 한국어와 러시아어 부분 사이에 '\n****\n' 문자열을 넣어주세요.
답변의 러시아와 독일어 부분 사이에 '\n====\n' 문자열을 넣어주세요.
영어 텍스트:
{content}
"""

# 프롬프트 정의
summary_prompt = PromptTemplate(template=summary_prompt_template,
                                input_variables=["content"])

# 체인 구성
summary_chain = summary_prompt | llm

# 요약 및 번역 수행
prompt_input = summary_prompt.format(content=english_text)  # 프롬프트 텍스트로 변환
original_answer = summary_chain.invoke(prompt_input)       # 텍스트 입력
print(original_answer.content)

# 결과 출력
print("\n요약 및 번역 결과:\n")
print(textwrap.fill(
    str(original_answer.content), width=50, replace_whitespace=True
    ).replace("\\n ", "\n\n"
    ).replace("----", "\n한국어:\n"
    ).replace("****", "\n러시아어:\n").replace("====", "\n독일어:\n"))


Good relationships are essential for a healthy and happy life, as revealed by the Harvard Study of Adult Development, which observed the lives of 724 men over 75 years. The research emphasizes that social connections are vital, and loneliness can be toxic, affecting individuals' happiness and health. Quality of relationships, rather than quantity, is crucial, with satisfying relationships leading to better outcomes as people age. Ultimately, investing time and energy into nurturing relationships is key to a fulfilling life.

----
좋은 관계는 건강하고 행복한 삶에 필수적이며, 이는 75년 동안 724명의 남성을 관찰한 하버드 성인 발달 연구에 의해 밝혀졌습니다. 연구는 사회적 연결이 중요하고 외로움이 건강과 행복에 부정적인 영향을 미친다고 강조합니다. 관계의 질이 양보다 중요하며, 만족스러운 관계가 나이가 들수록 더 나은 결과로 이어진다고 합니다. 결국, 관계를 키우는 데 시간과 에너지를 투자하는 것이 충만한 삶의 핵심입니다.

****
Хорошие отношения являются важными для здоровой и счастливой жизни, как показывает Исследование взрослых людей Гарварда, которое наблюдало за жизнью 724 мужчин на протяжении 75 лет. Исследование подчеркивает, что социальные связи им

In [ ]:
# 주요 요점 추출 프롬프트 템플릿 정의
key_points_prompt_template = """
당신은 간결하고 명확한 요점을 제공하는 전문가입니다.
이전 질문에 대한 답변은 다음과 같습니다: <{original_answer}>
이 한국어 답변을 3-5개의 주요 요점으로 정리해 주세요.
각 요점은 글머리 기호로 시작하는 한국어로만 작성해 주세요.
"""
key_points_prompt = PromptTemplate(template=key_points_prompt_template,
                                   input_variables=["original_answer"])

# 체인 구성 (RunnableSequence 사용)
key_points_chain = key_points_prompt | llm

# 요점 추출 수행
key_points = key_points_chain.invoke({"original_answer": original_answer})  # .invoke 사용

# 결과 출력
print("\n한국어 주요 요점:\n")
print(key_points.content)


한국어 주요 요점:

- 좋은 관계는 건강하고 행복한 삶에 필수적이다.
- 사회적 연결이 중요하며, 외로움은 건강과 행복에 부정적인 영향을 미친다.
- 관계의 질이 양보다 더 중요하며, 만족스러운 관계가 나이가 들수록 더 나은 결과를 가져온다.
- 관계를 유지하는 데 시간과 에너지를 투자하는 것이 충만한 삶의 핵심이다.
